In [1]:
%pip install azureml-widgets
%pip install azureml-train-automl-runtime==1.57.0
%pip install --upgrade azureml-sdk[notebooks,automl]

StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 12, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 84.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 110.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.7/548.7 KB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 KB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 KB 38.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.4
    Not uninstalling urllib3 at /home/trusted-service-user/cluster-env/env/lib/python3.10/site-packages, outside environment /nfs4/pyenv-6e5c7a19-8ad3-4440-983d-f55556a07b94
    Can't uninstall 'urllib3'. No files were found to uninstall.
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Not uninstalling typing-extensions at /home/trusted-service-user/cluster-env/env/lib/python3.10/site-packages, outside environment /nfs4/pyenv-6e5c7a19-8ad3-4440-98

In [20]:
from sklearn.model_selection import train_test_split
import pandas as pd
from azureml.core import Dataset

StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 36, Finished, Available, Finished)

In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config("Users/odl_user_270515/config.json")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 18, Finished, Available, Finished)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code LDAWC5MUL to authenticate.
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-270515
Azure region: westeurope
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-270515


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "mycluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
vm_size = "Standard_D2_V2"
compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=4)
#Create the compute cluster
compute_target = ComputeTarget.create(ws,cluster_name, compute_config)
#wait for the compute ready
compute_target.wait_for_completion(show_output=True)

StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 20, Finished, Available, Finished)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
# ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {"C": uniform(0.05, 0.1),
    "max_iter" : choice(16, 32, 64, 128)}
)

# Specify a Policy
# policy = ### YOUR CODE HERE ###
# Specify a Policy
policy = BanditPolicy(slack_factor=0.2, evaluation_interval=1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
# sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')
sklearn_env = Environment.from_conda_specification(name='sklearn-env', 
file_path='Users/odl_user_270515/conda_dependencies.yml')


# Create a ScriptRunConfig Object to specify the configuration details of your training job
# src = ### YOUR CODE HERE ###
src = ScriptRunConfig(
    source_directory=".",
    script="Users/odl_user_270515/train.py",
    environment=sklearn_env,
    compute_target=compute_target
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=16,
    max_concurrent_runs=8
)




StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 21, Finished, Available, Finished)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
experiment = Experiment(workspace=ws, name="manual-hyperdive")
experiment_run = experiment.submit(hyperdrive_config)
RunDetails(experiment_run).show()
experiment_run.wait_for_completion(show_output=True)

StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 22, Finished, Available, Finished)

Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory


Initializing logging file for interpret-community


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_c4a12be0-8384-4416-be0b-2be4d4d06e30
Web View: https://ml.azure.com/runs/HD_c4a12be0-8384-4416-be0b-2be4d4d06e30?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-270515/workspaces/quick-starts-ws-270515&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-12-11T10:13:08.0184238Z][GENERATOR][DEBUG]Sampled 8 jobs from search space 
[2024-12-11T10:13:08.5382124Z][SCHEDULER][INFO]Scheduling job, id='HD_c4a12be0-8384-4416-be0b-2be4d4d06e30_0' 
[2024-12-11T10:13:08.7619697Z][SCHEDULER][INFO]Scheduling job, id='HD_c4a12be0-8384-4416-be0b-2be4d4d06e30_3' 
[2024-12-11T10:13:08.6341254Z][SCHEDULER][INFO]Scheduling job, id='HD_c4a12be0-8384-4416-be0b-2be4d4d06e30_1' 
[2024-12-11T10:13:08.8243555Z][SCHEDULER][INFO]Scheduling job, id='HD_c4a12be0-8384-4416-be0b-2be4d4d06e30_6' 
[2024-12-11T10:13:08.8221996Z][SCHEDULER][INFO]Scheduling job, id='HD_c4a12be0-8384-4416-be0b-2be4d4d06e30_7' 
[2024-12-11T10:13:08.7636758Z]

{'runId': 'HD_c4a12be0-8384-4416-be0b-2be4d4d06e30',
 'target': 'mycluster',
 'status': 'Completed',
 'startTimeUtc': '2024-12-11T10:13:06.361255Z',
 'endTimeUtc': '2024-12-11T10:31:17.15019Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '6de890be-061c-47f5-ad03-57ccd42beb03',
  'user_agent': 'python/3.10.6 (Linux-4.15.0-1178-azure-x86_64-with-glibc2.27) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.59.0',
  'best_child_run_id': 'HD_c4a12be0-8384-4416-be0b-2be4d4d06e30_5',
  'score': '0.9084977238239758',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_c4a12be0-8384-4416-be0b-2be4d4d06e30_5'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-trai

In [7]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_hyper_run = experiment_run.get_best_run_by_primary_metric()
print(best_hyper_run.get_metrics())

StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 23, Finished, Available, Finished)

{'Regularization Strength:': 0.05430529741600926, 'Max iterations:': 64, 'Accuracy': 0.9084977238239758}


In [8]:
model = best_hyper_run.register_model(model_name = "best_hyper_run_model",
model_path="outputs/model.joblib")

StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 24, Finished, Available, Finished)

AutoML

In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
data = TabularDatasetFactory.from_delimited_files(url)

StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 25, Finished, Available, Finished)

In [10]:
data

StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 26, Finished, Available, Finished)

{
  "definition": "EnginelessDataflow:\n---\ntype: mltable\npaths:\n  - pattern: \"wasbs://automl-sample-notebook-data@automlsamplenotebookdata.blob.core.windows.net/bankmarketing_train.csv\"\ntransformations:\n  - read_delimited:\n      path_column: Path\n      include_path_column: false\n      encoding: utf8\n      support_multi_line: false\n      delimiter: \",\"\n      empty_as_string: false\n      partition_size: 20971520\n      header: all_files_same_headers\n      infer_column_types: false\n  - convert_column_types:\n      - columns: duration\n        column_type: int\n      - columns: education\n        column_type: string\n      - columns: pdays\n        column_type: int\n      - columns: campaign\n        column_type: int\n      - columns: previous\n        column_type: int\n      - columns: cons.conf.idx\n        column_type: float\n      - columns: job\n        column_type: string\n      - columns: euribor3m\n        column_type: float\n      - columns: poutcome\n        co

In [12]:
from Users.odl_user_270515.train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(data)

StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 28, Finished, Available, Finished)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': '930d601d-be3d-4946-9f85-ab1427f18050'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': '930d601d-be3d-4946-9f85-ab1427f18050', 'run_id': '930d601d-be3d-4946-9f85-ab1427f18050'}


In [14]:
#Split train and test set with ratio 80:20
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
y_train = y_train.to_frame()

StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 30, Finished, Available, Finished)

In [17]:
# Format the training data
train_data = pd.concat([x_train, y_train], axis=1)
train_data.head()

StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 33, Finished, Available, Finished)

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
26465,52,1,0,1,0,8,3,13,14,999,...,0,0,0,0,1,0,0,0,0,0
1387,41,1,0,1,0,6,4,84,3,999,...,1,1,0,0,0,0,0,0,0,0
22628,27,0,0,0,0,7,1,331,2,999,...,0,0,0,0,0,0,1,0,0,0
8336,58,1,0,1,0,6,2,137,3,999,...,1,1,0,0,0,0,0,0,0,0
19898,36,0,0,1,0,5,1,143,4,999,...,1,1,0,0,0,0,0,0,0,0


In [18]:
#Upload training data to datastore
datastore = ws.get_default_datastore()

StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 34, Finished, Available, Finished)

In [19]:
# Save training data to csv
train_data.to_csv("training/training_data.csv", index=False)

StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 35, Finished, Available, Finished)

In [21]:
datastore.upload(src_dir='training/', target_path='data/')

StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 37, Finished, Available, Finished)

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.
Uploading an estimated of 1 files
Uploading training/training_data.csv
Uploaded training/training_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_84d2318be6994cd08766da74d8b53748

In [22]:
#Upload the training_data.csv to dataset
data_on_datasets = Dataset.Tabular.from_delimited_files(path=[(datastore, ('data/training_data.csv'))])

StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 38, Finished, Available, Finished)

In [24]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=data_on_datasets,
    label_column_name='y',
    n_cross_validations=4,
    max_concurrent_iterations=4,
    compute_target=compute_target)


StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 40, Finished, Available, Finished)

In [25]:
# Submit your automl run

### YOUR CODE HERE ###
exp_automl = Experiment(ws, 'exp_automl')
automl_run = exp_automl.submit(automl_config, show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 41, Finished, Available, Finished)

Submitting remote run.
No run_configuration provided, running on mycluster with default configuration
Running on remote compute: mycluster


Experiment,Id,Type,Status,Details Page,Docs Page
exp_automl,AutoML_a6a399d1-caf0-45a2-a453-f9f783b2f4a3,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Nu

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
exp_automl,AutoML_a6a399d1-caf0-45a2-a453-f9f783b2f4a3,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|2951                          |True                            |26360                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_a6a399d1-caf0-45a2-a453-f9f783b2f4a3',
 'target': 'mycluster',
 'status': 'Completed',
 'startTimeUtc': '2024-12-11T10:45:58.790569Z',
 'endTimeUtc': '2024-12-11T11:31:27.115781Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'mycluster',
  'AMLSettingsJsonString': '{"path":null,"name":"exp_automl","subscription_id":"3d1a56d2-7c81-4118-9790-f85d1acf0c77","resource_group":"aml-quickstarts-270515","workspace_name":"quick-starts-ws-270515","region":"westeurope","compute_target":"mycluster","spark_service":

In [26]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_run, best_automl_model = automl_run.get_output()

StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 42, Finished, Available, Finished)

[11:43:14] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:1203: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[11:43:14] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:888: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
[11:43:14] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, pl

In [27]:
best_automl_run.register_model(model_name="best_automl_run_model.pkl",
model_path='./outputs/')

StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 43, Finished, Available, Finished)

Model(workspace=Workspace.create(name='quick-starts-ws-270515', subscription_id='3d1a56d2-7c81-4118-9790-f85d1acf0c77', resource_group='aml-quickstarts-270515'), name=best_automl_run_model.pkl, id=best_automl_run_model.pkl:1, version=1, tags={}, properties={})

In [28]:
print(best_automl_model._final_estimator)

StatementMeta(d70b072d-1dad-4c36-b691-f2dd55bbe685, 0, 44, Finished, Available, Finished)

PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('32', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bylevel=0.6, colsample_bytree=1, eta=0.001, gamma=0, max_depth=6, max_leaves=15, n_estimators=800, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0, reg_lambda=2.5, subsample=1, tree_method='auto'))], verbose=False)), ('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('18', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False))

In [ ]:
#delete compute_target
compute_target.delete()